In [13]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

In [55]:
df = pd.read_csv('Online_Sales.csv')
discount = pd.read_csv('Discount_Coupon.csv')
tax = pd.read_excel('Tax_amount.xlsx')

# RFM Segmentation

In [39]:
df.head()

,CustomerID,Transaction_ID,Transaction_Date,Product_SKU,Product_Description,Product_Category,Quantity,Avg_Price,Delivery_Charges,Coupon_Status
0,17850,16679,20190101,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.5,Used
1,17850,16680,20190101,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.5,Used
2,17850,16681,20190101,GGOEGFKQ020399,Google Laptop and Cell Phone Stickers,Office,1,2.05,6.5,Used
3,17850,16682,20190101,GGOEGAAB010516,Google Men's 100% Cotton Short Sleeve Hero Tee...,Apparel,5,17.53,6.5,Not Used
4,17850,16682,20190101,GGOEGBJL013999,Google Canvas Tote Natural/Navy,Bags,1,16.50,6.5,Used


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52924 entries, 0 to 52923
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CustomerID           52924 non-null  int64  
 1   Transaction_ID       52924 non-null  int64  
 2   Transaction_Date     52924 non-null  int64  
 3   Product_SKU          52924 non-null  object 
 4   Product_Description  52924 non-null  object 
 5   Product_Category     52924 non-null  object 
 6   Quantity             52924 non-null  int64  
 7   Avg_Price            52924 non-null  float64
 8   Delivery_Charges     52924 non-null  float64
 9   Coupon_Status        52924 non-null  object 
dtypes: float64(2), int64(4), object(4)
memory usage: 4.0+ MB


## Recency 

In [56]:
df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date'], format='%Y%m%d')
df['Transaction_Date'].max()

Timestamp('2019-12-31 00:00:00')

In [31]:
now = pd.to_datetime('2019, 12, 31')

In [20]:
recency = df.groupby('CustomerID', as_index=False)[['Transaction_Date']].max()

In [21]:
recency.columns = ['CustomerID','Last_purchase_date']

In [23]:
recency.head()

,CustomerID,Last_purchase_date
0,12346,2019-09-15
1,12347,2019-11-02
2,12348,2019-10-19
3,12350,2019-12-14
4,12356,2019-09-15


In [32]:
recency['recency'] = recency['Last_purchase_date'].apply(lambda x: (now - x).days)

In [77]:
recency.head()

,CustomerID,recency
0,12346,107
1,12347,59
2,12348,73
3,12350,17
4,12356,107


In [34]:
recency.drop('Last_purchase_date', axis=1, inplace=True)

In [36]:
recency.head()

,CustomerID,recency
0,12346,107
1,12347,59
2,12348,73
3,12350,17
4,12356,107


## Frequency

In [44]:
frequency= df.groupby('CustomerID', as_index=False)[['Transaction_ID']].count()

In [47]:
frequency.columns = ['CustomerID','Frequency']

In [78]:
frequency.head()

,CustomerID,Frequency
0,12346,2
1,12347,60
2,12348,23
3,12350,17
4,12356,36


## Monetary 

In [57]:
sale_discount = pd.merge(left =df, right=discount, how='left', left_on='Product_Category', right_on='Product_Category ')
sale_dtax = pd.merge(left=sale_discount, right=tax, how='left', left_on='Product_Category' , right_on='Product_Category')

In [58]:
sale_dtax['invoice_value'] = ((sale_dtax['Quantity']*sale_dtax['Avg_Price'])*(1-(sale_dtax['Discount_pct']/100))*(1+sale_dtax['GST']))+sale_dtax['Delivery_Charges']

In [63]:
monetary = sale_dtax.groupby('CustomerID', as_index=False)[['invoice_value']].sum()

In [75]:
monetary.columns = ['CustomerID','monetary']

In [79]:
monetary.head()

,CustomerID,monetary
0,12346,2142.60672
1,12347,160334.29440
2,12348,18164.71968
3,12350,16014.56832
4,12356,22954.48608


In [113]:
## Creating RFM table
fm = frequency.merge(monetary, on='CustomerID')
rfm = fm.merge(recency, on='CustomerID')

In [114]:
rfm.head()

,CustomerID,Frequency,monetary,recency
0,12346,2,2142.60672,107
1,12347,60,160334.29440,59
2,12348,23,18164.71968,73
3,12350,17,16014.56832,17
4,12356,36,22954.48608,107


In [115]:
rfm.set_index('CustomerID', inplace=True)

In [117]:
## Creating Quartiles
quantiles = rfm.quantile(q=[0.33,0.67])

In [118]:
quantiles

,Frequency,monetary,recency
0.33,13.0,11483.517782,78.00
0.67,35.0,36479.872656,187.89


In [121]:
rfm['r_seg'] = np.where(rfm.recency <= 78, 'Active', np.where(rfm.recency <= 187, 'At Risk','Churn'))

In [123]:
rfm['f_seg'] = np.where(rfm.Frequency <= 13, 1,np.where(rfm.Frequency <=35, 2, 3))
rfm['m_seg'] = np.where(rfm.monetary <= 11483, 1,np.where(rfm.monetary <=36479, 2, 3))

In [124]:
rfm

,Frequency,monetary,recency,r_seg,f_seg,m_seg
CustomerID,,,,,,
12346,2,2142.60672,107,At Risk,1,1
12347,60,160334.29440,59,Active,3,3
12348,23,18164.71968,73,Active,2,2
12350,17,16014.56832,17,Active,2,2
12356,36,22954.48608,107,At Risk,3,2
...,...,...,...,...,...,...
18259,7,8780.07840,270,Churn,1,1
18260,40,32184.68736,87,At Risk,3,2
18269,8,1766.02944,194,Churn,1,1


In [125]:
rfm['fm_seg'] = np.where(((rfm.f_seg == 3) & (rfm.m_seg ==3)) , 'Platinum',
                          np.where((((rfm.f_seg == 3) & (rfm.m_seg ==2)) | ((rfm.f_seg == 2) & (rfm.m_seg ==3))) , 'Gold',
                          np.where((((rfm.f_seg == 1) & (rfm.m_seg ==3)) | ((rfm.f_seg == 3) & (rfm.m_seg ==1)) | ((rfm.f_seg == 2) & (rfm.m_seg ==2))), 'Silver', 'Standard' )))

In [127]:
rfm.head()

,Frequency,monetary,recency,r_seg,f_seg,m_seg,fm_seg
CustomerID,,,,,,,
12346,2,2142.60672,107,At Risk,1,1,Standard
12347,60,160334.29440,59,Active,3,3,Platinum
12348,23,18164.71968,73,Active,2,2,Silver
12350,17,16014.56832,17,Active,2,2,Silver
12356,36,22954.48608,107,At Risk,3,2,Gold


In [132]:
## Active Platinum customers
rfm[(rfm.fm_seg == 'Platinum') & (rfm.r_seg =='Active')]

,Frequency,monetary,recency,r_seg,f_seg,m_seg,fm_seg
CustomerID,,,,,,,
12347,60,160334.29440,59,Active,3,3,Platinum
12377,77,112224.98304,40,Active,3,3,Platinum
12431,44,62938.39680,40,Active,3,3,Platinum
12471,128,159513.23904,15,Active,3,3,Platinum
12472,126,140425.38240,62,Active,3,3,Platinum
...,...,...,...,...,...,...,...
18077,63,61876.27296,73,Active,3,3,Platinum
18125,83,120849.14400,48,Active,3,3,Platinum
18145,42,46115.77056,33,Active,3,3,Platinum


In [135]:
## Customer Profiling
col = ['Frequency','monetary','recency']
profiling_output= rfm.groupby(['r_seg', 'fm_seg']).apply(lambda x: x[col].mean()).T

In [136]:
profiling_output

r_seg            Active                                            \
fm_seg             Gold       Platinum        Silver     Standard   
Frequency     31.529412      94.492228     22.181818     8.681818   
monetary   42524.882259  109892.435376  24566.281152  8991.133331   
recency       34.117647      35.145078     35.863636    35.977273   

r_seg           At Risk                                           \
fm_seg             Gold      Platinum        Silver     Standard   
Frequency     41.652174     86.360902     24.360000     8.522936   
monetary   34095.155082  85627.156389  22172.332464  6496.570503   
recency      121.739130    125.233083    134.960000   133.830275   

r_seg             Churn                                           
fm_seg             Gold      Platinum        Silver     Standard  
Frequency     34.526316     68.853933     21.654867     7.795918  
monetary   39186.047848  74931.976902  21378.606983  6766.041345  
recency      263.894737    271.123596    256.132743   271.738776

# Value based segmentation

In [139]:
sale_dtax.head()

,CustomerID,Transaction_ID,Transaction_Date,Product_SKU,Product_Description,Product_Category,Quantity,Avg_Price,Delivery_Charges,Coupon_Status,Month,Product_Category,Coupon_Code,Discount_pct,GST,invoice_value
0,17850,16679,2019-01-01,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.5,Used,Jan,Nest-USA,ELEC10,10.0,0.1,158.6729
1,17850,16679,2019-01-01,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.5,Used,Feb,Nest-USA,ELEC20,20.0,0.1,141.7648
2,17850,16679,2019-01-01,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.5,Used,Mar,Nest-USA,ELEC30,30.0,0.1,124.8567
3,17850,16679,2019-01-01,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.5,Used,Apr,Nest-USA,ELEC10,10.0,0.1,158.6729
4,17850,16679,2019-01-01,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,153.71,6.5,Used,May,Nest-USA,ELEC20,20.0,0.1,141.7648


In [141]:
value = sale_dtax.groupby('CustomerID')[['invoice_value']].sum()

In [143]:
value.columns = ['Monetary']

In [177]:
value

,CustomerID,Frequency,monetary,recency,segment
0,12346,2,2142.60672,107,Standard
1,12347,60,160334.29440,59,Platinum
2,12348,23,18164.71968,73,Silver
3,12350,17,16014.56832,17,Silver
4,12356,36,22954.48608,107,Gold
...,...,...,...,...,...
1463,18259,7,8780.07840,270,Silver
1464,18260,40,32184.68736,87,Gold
1465,18269,8,1766.02944,194,Standard
1466,18277,1,3218.88000,69,Standard


In [158]:
fm = frequency.merge(monetary, on='CustomerID')
value = fm.merge(recency, on='CustomerID')

In [159]:
value

,CustomerID,Frequency,monetary,recency
0,12346,2,2142.60672,107
1,12347,60,160334.29440,59
2,12348,23,18164.71968,73
3,12350,17,16014.56832,17
4,12356,36,22954.48608,107
...,...,...,...,...
1463,18259,7,8780.07840,270
1464,18260,40,32184.68736,87
1465,18269,8,1766.02944,194
1466,18277,1,3218.88000,69


In [160]:
segment = value.quantile([0.25,0.50,0.75,1])

In [161]:
segment

,CustomerID,Frequency,monetary,recency
0.25,13830.50,10.0,8238.89136,55.0
0.50,15300.00,21.0,21228.61632,131.0
0.75,16882.25,46.0,46534.26960,220.0
1.00,18283.00,695.0,879262.04448,364.0


In [164]:
value['segment'] = np.where(value.monetary <= 8238, 'Standard', np.where(value.monetary <= 21228, 'Silver', np.where(value.monetary <=46534,'Gold','Platinum'))) 

In [168]:
value

,CustomerID,Frequency,monetary,recency,segment
0,12346,2,2142.60672,107,Standard
1,12347,60,160334.29440,59,Platinum
2,12348,23,18164.71968,73,Silver
3,12350,17,16014.56832,17,Silver
4,12356,36,22954.48608,107,Gold
...,...,...,...,...,...
1463,18259,7,8780.07840,270,Silver
1464,18260,40,32184.68736,87,Gold
1465,18269,8,1766.02944,194,Standard
1466,18277,1,3218.88000,69,Standard


In [166]:
value.segment.value_counts()

Silver      367
Standard    367
Platinum    367
Gold        367
Name: segment, dtype: int64

In [172]:
## Platinum customers
value[(value.segment == 'Platinum')]

,CustomerID,Frequency,monetary,recency,segment
1,12347,60,160334.29440,59,Platinum
6,12370,91,81511.92960,189,Platinum
8,12377,77,112224.98304,40,Platinum
9,12383,69,57629.60352,99,Platinum
25,12431,44,62938.39680,40,Platinum
...,...,...,...,...,...
1450,18202,23,51860.57280,23,Platinum
1453,18223,60,51137.44320,87,Platinum
1459,18239,52,69320.04960,34,Platinum
1460,18245,55,84508.10688,33,Platinum


In [173]:
#profiling
col=['monetary','recency','Frequency']

In [176]:
profiling_output= value.groupby(['segment']).apply(lambda x: x[col].mean()).T
profiling_output

segment,Gold,Platinum,Silver,Standard
monetary,32342.917063,102877.217089,14074.860004,3535.748568
recency,130.754768,111.111717,160.193460,175.108992
Frequency,32.386921,89.820163,15.926431,6.073569
